# Tutorial 2: System identification
In this tutorial we will calibrate our double pendulum model for our model-based controllers. To do so, we will run a few experiments and use the data to approximate the physical properties of our robot.

**Pre-requisites**

Knowledge of the dynamics of the double pendulum, basic filters.

**Goals**

Obtaining a good estimation of the properties of the physical double pendulum.

This notebook is organized as follows:

    1. Introduction
    2. Problem statement
    3. Measurement
    4. System identification

Run the next cell to make sure you have all the necessary dependencies installed.

In [ ]:
!pip install -r requirements.txt

In [ ]:
# imports
import numpy as np
import pandas as pd
import os
from datetime import datetime
from double_pendulum.model.plant import DoublePendulumPlant
from double_pendulum.simulation.simulation import Simulator
from double_pendulum.utils.plotting import plot_timeseries
from double_pendulum.system_identification.sys_id import (run_system_identification,
                                                          run_system_identification_nl)
from double_pendulum.utils.csv_trajectory import save_trajectory, concatenate_trajectories
from double_pendulum.controller.gravity_compensation.gravity_compensation_controller import GravityCompensationController
from double_pendulum.controller.pid.trajectory_pid_controller import TrajPIDController

%matplotlib inline
from IPython.display import HTML
import matplotlib as mpl
mpl.rcParams['animation.writer'] = "pillow"
import matplotlib.pyplot as plt

## 1. Introduction

There are three main approaches to system identification:

- Computer-Aided Design:

Many CAD programs can quickly and accurately calculate the dimensions and inertial parameters of the parts that make up our robots. However, this method often does not take into account components like electronics or properties like motor inertia. Additionally, a real prototype may suffer from important deviations from the original design.

- Measuring by hand:

The advantage of this method is that the measurements are made on the real system, avoiding any deviation between design and reality. However, the process of weighing and measuring parts can take a long time, especially if disassembling is required. Additionally, the quality of the measurements is dependent on the tools available and the human factor. Unlike the previous and next options, this method only accurately provides masses and dimensions, leaving moments of inertia to be infered from these parameters.

- Analyzing data from the real robot:

This last method includes a wide variety of approaches. It consists in comparing data predicted by the model to the data measured by the robot during operation. In our case, this means comparing the predicted torque to the torque that is really being supplied to the robot and then adapting the model accordingly. This method can approxiamte many parameters of the robot's model, including friction in many cases.

Because some accurate measurements, like the dimensions of the double pendulum, are already available to us, we only need to identify the following parameters:

- $I_1$: Moment of inertia of the first link.
- $I_2$: Moment of inertia of the second link.
- $b_1$: Shoulder viscous friction coefficient.
- $b_2$: Elbow viscous friction coefficient.
- $\mu_1$: Coulomb friction on the shoulder.
- $\mu_2$: Coulomb friction on the elbow.
- $m_1$: Mass of the first link.
- $m_2$: Mass of the second link.
- $r_2$: Distance from the elbow joint to the second link's center of mass

<img src="double_pendulum_coords.png" width="50%" align="" />


## 2. Problem formulation

The parameters that we wish to quantify are related to the dynamics of the double pendulum. Of these, some are inertial (mass, location of the center of mass, and moment of inertia) and others are related to various types of friction. Before we can run our experiment, first we must assemble a **parameter vector**($\mathbf{\Phi}$). The parameter vector, when multiplied by the **regressor matrix**($\mathbf{Y}$) obeys the following equality:

$$Y(\mathbf{q}, \dot{\mathbf{q}}, \ddot{\mathbf{q}}) \mathbf{\Phi}= \mathbf{\tau}
$$

The general parameter vector for the inertial parameters of a body in SE(3) is

$$ \mathbf{\Phi}_i = \left[ \begin{array}{cccccccccc} m & r_x m & r_y m & r_z m & I_{xx} & I_{xy} & I_{xz} & I_{yy} & I_{yz} & I_{zz} \end{array}\right]^T$$

Since the double pendulum moves in a 2D plane and we have reliable measurements for some parameters, the number of inertial parameters is significantly reduced. On top of the inertial parameters, we also include the friction parameters:

$$ \mathbf{\Phi} = \left[ \begin{array}{cccccccccc} r_1 m_1 & I_1 & \mu_1 & b_1 & r_2 m_2 & m_2 &I_2 & \mu_2 & b_2 \end{array}\right]$$

Next, we will have to rearrange the equations of motion to obtain the regressor matrix.

$$ M(\mathbf{q})\ddot{\mathbf{q}} + C(\mathbf{q}, \dot{\mathbf{q}})\dot{\mathbf{q}} + G(\mathbf{q}) +F(\dot{\mathbf{q}}) = \mathbf{\tau} $$

We may do so by differentiating the equations with respect to their corresponding parameter:

$$ \mathbf{Y} = \left[ \begin{array}{cccc} \frac{\partial Q_1}{\partial \mathbf{\Phi}_1} & \frac{\partial Q_1}{\partial \mathbf{\Phi}_2} & \dots & \frac{\partial Q_1}{\partial \mathbf{\Phi}_m} \\ \frac{\partial Q_2}{\partial \mathbf{\Phi}_1} & \frac{\partial Q_2}{\partial \mathbf{\Phi}_2} & \dots & \frac{\partial Q_2}{\partial \mathbf{\Phi}_m} \\ \vdots & \vdots &\ddots & \vdots \\ \frac{\partial Q_n}{\partial \mathbf{\Phi}_1} & \frac{\partial Q_n}{\partial \mathbf{\Phi}_2} & \dots & \frac{\partial Q_n}{\partial \mathbf{\Phi}_m}\end{array} \right]$$

Where $Q_i$ is the left hand side of the equation of motion for coordinate $i$ and $\mathbf{\Phi}_i$ is the $i$-th parameter in the parameter vector. Notice that $\mathbf{Y}$ is not necessarily square. In fact, $\mathbf{Y}$ is most often non-square.



Just as a reminder these are the terms of the manipulator equation that describes the motion of the double pendulum. The mass matrix is given by (with $s_1 = \sin(q_1), c_1 = \cos(q_1), \ldots$):
$$
  M =
    \left[ {\begin{array}{cc}
          I_1 + I_2 + l_1^2m_2 + 2l_1m_2r_2c_2 + g_r^2I_r + I_r &   I_2 + l_1m_2r_2c_2 \\
          I_2 + l_1m_2r_2c_2 &   I_2 \\
    \end{array}} \right]
$$

The Coriolis Matrix is:
$$
    C =
    \left[\begin{matrix}
    - 2 \dot{q}_2 l_{1} m_{2} r_{2} s_2 & - \dot{q}_2 l_{1} m_{2} r_{2} s_2\\
    \dot{q}_1 l_{1} m_{2} r_{2} s_2 &   0
    \end{matrix}\right]
$$

and the gravity vector:
$$
    G =
    \left[ \begin{matrix}
    - g m_{1} r_{1} s_1 - g m_{2} \left(l_{1}
s_1 + r_{2} s_{1+2} \right)\\
    - g m_{2} r_{2} s_{1+2}
    \end{matrix} \right]
$$

and the friction term:
$$
    F = 
    \left[\begin{matrix}
    b \dot{q}_1 + \mu \arctan(100 \dot{q}_1)\\
    b \dot{q}_2 + \mu \arctan(100 \dot{q}_2)
    \end{matrix}\right]
$$

where

$$ q = (q_1, q_2)^T, x = (q, \dot{q})^T, \tau = (\tau_1, \tau_2)^T $$

The goal of the next few operations is to retrieve $\mathbf{\Phi}$ from the result of our experiment. One simple way of achieving this is to use a least squares approach. This means that we will need to solve the following problem:

$$ \min_{\mathbf{\Phi}} \| \mathbf{Y}(\mathbf{q}, \dot{\mathbf{q}}, \ddot{\mathbf{q}}) \mathbf{\Phi} - \tau \|^2$$

## 3. Measurement

To get some experimental torque and state data, we will run a PID controller following a trajectory. Using a PID means we don't need to have any information about dynamics of the system to run this test.
As a short reminder, this is the expression for the control law of a PID (Proportional-Integral-Derivative) controller:
$$
\tau(t, x) = K_p \bar{\mathbf{x}}(t) + K_i \int_0^t \bar{\mathbf{x}}(t') \text{d}t' + K_d \dot{\bar{\mathbf{x}}} \\
$$

where $K_p$, $K_i$, and $K_d$ are the proportional, integral, and derivative gain, respectively. $\bar{\mathbf{x}}$ is defined by the following expression:

$$
\bar{\mathbf{x}}(t) = \mathbf{x}_{goal}(t) - \mathbf{x}(t)
$$

Run the experiment. Running it generates a results file containing the torques that were supplied during the motion, as well as the measured joint positions, velocities, and accelerations. Notice that these are the variables that we need to compute the regressor matrix for at every instant.

Run the next cell to set up a PID controller for the first experiment, Notice that we save the positions, velocities, and accelerations of the joints of the double pendulum. These are the variables needed to compute $\mathbf{Y}$ at every instant.

In [ ]:
motors = [403, 379] # REPLACE WITH YOUR MOTOR IDS [SHOULDER, ELBOW]
# Set up provisional plant

dt=0.001
tf=10
torque_limit= [0.15, 0.15]
# Plant Model Setup
plant = DoublePendulumPlant(
    mass=[0.131, 0.064],
    length=[0.05, 0.05] ,
    com=[0.046, 0.048],
    gravity=9.81,
    inertia=[0.000277196, 0.000147456],
    damping=[0.002137, 0.0003603],
    torque_limit=torque_limit,
)
sim = Simulator(plant=plant)

# Trajectory Setup
csv_path = "data/trajectory-pos-50.csv"

data = pd.read_csv(csv_path)
time_traj = np.asarray(data["time"])
pos1_traj = np.asarray(data["pos1"])
pos2_traj = np.asarray(data["pos2"])
vel1_traj = np.asarray(data["vel1"])
vel2_traj = np.asarray(data["vel2"])
acc1_traj = np.asarray(data["acc1"])
acc2_traj = np.asarray(data["acc2"])

T_des = time_traj
X_des = np.asarray([pos1_traj, pos2_traj, vel1_traj, vel2_traj]).T
ACC_des = np.asarray([acc1_traj, acc2_traj]).T

# Controller parameters
Kp = 0.14
Ki = 0.0
Kd = 0.002
dt=0.003333
tf=30
trajPID_controller = TrajPIDController(csv_path=csv_path, dt=dt, torque_limit=torque_limit)
trajPID_controller.set_parameters(Kp=Kp, Ki=Ki, Kd=Kd)
sim.reset()
sim.set_desired_traj(T_des, X_des)

You may optionally run the next cell to make sure your controller is properly set up.

In [ ]:
# Simulation
x0=[0.0,0,0,0]
trajPID_controller.init()
trajPID_controller.counter=0
T, X, U, anim = sim.simulate_and_animate(
    t0=0.0,
    x0=x0,
    tf=tf,
    dt=dt,
    controller=trajPID_controller,
    integrator="runge_kutta",
    plot_inittraj=True,
)

html = HTML(anim.to_jshtml())
display(html)
plt.close()

acc = np.diff(np.asarray(X).T[2:, :]).T / dt
acc = np.insert(acc, 0, [0, 0], axis=0)

plot_timeseries(
    T,
    X,
    U,
    acc,
    plot_acc=True,
    plot_energy=False,
    pos_y_lines=[0.0, np.pi],
    tau_y_lines=[-torque_limit[0], torque_limit[0]],
    T_des=T_des,
    X_des=X_des,
    ACC_des=ACC_des,
)

Run the next cell to execute the experiment. The motion should be similar to that shown in the simulation. Be sure to not interfere with the pendulum. 

In [ ]:
trajPID_controller.counter=0
# Hardware Implementation
trajPID_controller = TrajPIDController(csv_path=csv_path, dt=dt, torque_limit=torque_limit)
trajPID_controller.set_parameters(Kp=Kp, Ki=Ki, Kd=Kd)
T_exp, X_exp, U_exp, U_des_exp = sim.run_experiment(
    tf=tf,
    dt=dt,
    controller=trajPID_controller,
    experiment_type="DoublePendulum",
    motors = motors # [shoulder, elbow]
)

# Plotting
plot_timeseries(
    T=T_exp,
    X=X_exp,
    U=U_exp,
    T_des=T_exp,
    X_des=X_des,
    U_des=U_des_exp,
    pos_y_lines=[0.0, np.pi],
    tau_y_lines=[-torque_limit[1], torque_limit[1]],
)

You will notice that there is considerable noise in the results, especially for the velocity and acceleration of the elbow joint. So before solving the lest squares problem, we need to clean up the data. This is a crucial step, since noise will likely have a big impact on the accuracy of our system identification. For this case, we will use a Butterworth filter. Run  the next cell to clean the data and plot the filtered velocities, accelerations, and torques.

In [ ]:
# Get data files
measured_csv_data = np.array([T_exp, np.asarray(X_exp).T[0], np.asarray(X_exp).T[1], np.asarray(X_exp).T[2], np.asarray(X_exp).T[3], np.asarray(U_exp).T[0],np.asarray(U_exp).T[1]]).T
np.savetxt("data/sysid_modelbased_exp_data.csv", measured_csv_data, delimiter = ',', header="time,pos_meas1,pos_meas2,vel_meas1,vel_meas2,tau_meas1,tau_meas2", comments="")

We can now solve the least squares problem. To do so, we use the data from the complete experiment by stacking regressor matrices as such:

$$ \min_{\mathbf{\Phi}} \lVert \left[ \begin{array}{c} \mathbf{Y}(\mathbf{q}_0, \dot{\mathbf{q}}_0, \ddot{\mathbf{q}}_0) \\ \mathbf{Y}(\mathbf{q}_1, \dot{\mathbf{q}}_1, \ddot{\mathbf{q}}_1) \\ \vdots \\ \mathbf{Y}(\mathbf{q}_N, \dot{\mathbf{q}}_N, \ddot{\mathbf{q}}_N) \end{array} \right] \mathbf{\Phi} - \left[ \begin{array}{c} \tau_0 \\ \tau_1 \\ \vdots \\ \tau_N \end{array} \right] \rVert^2$$

## 4. System Identification

Before we can solve the least squares problem, we will need to process the experimental data to remove as much noise as possible, since it will contaminate our results otherwise.

In [ ]:
#from double_pendulum.system_identification.sys_id import run_system_identification, run_system_identification_nl
# saving
from double_pendulum.system_identification.sys_id import run_system_identification, run_system_identification_nl
timestamp = datetime.today().strftime("%Y%m%d-%H%M%S")
save_dir = os.path.join("data", "system_identification", timestamp)
os.makedirs(save_dir)

In [ ]:
measured_data_csv=[
    "data/sysid_modelbased_exp_data.csv",
]
T, X, U = concatenate_trajectories(measured_data_csv,
                                   with_tau=True)
full_csv_path = os.path.join(save_dir, "full_trajectory.csv")
save_trajectory(full_csv_path, T, X, U)

In the next cell, we provide some information that we know about our system. The most important are:
- Gravity ($g$): 9.81m/s$^2$
- Distance between joints of the double pendulum ($l_1$): 0.05 m
- Motor inertia ($I_r$): 6.1e-06 kg/m$^2$

In [ ]:
# fixed model parameters (will not be fitted)
fixed_mpar = {"g": 9.81,
              "gr": 1,
              "l1": 0.05,
              "l2": 0.05,
              "Ir": 61e-06
             }

### Think-Pair-Share

Next, we define the parameter vector and the initial guess for the least squares problem. Pick initial the initial guess and bounds for each of the unknown properties of the system. Lastly, use the inital guesses of the unknowns to fill out the initial guess of $\mathbf{\Phi}$.

In [ ]:
variable_mpar = ["m1r1", "I1", "cf1", "b1",
                 "m2r2", "m2", "I2", "cf2", "b2"]

## Your code here!

m1 = 0.0
m2 = 0.0
I1 = 0.0
I2 = 0.0
r1=0.0
r2=0.0
b1=0.0
b2=0.0
cf1=0.0
cf2=0.0

bounds = np.array([[0.0, 0.0],      # r1*m1
                   [0.0, 0.0],       # I1
                   [0.0, 0.0],       # cf1
                   [0.0, 0.0],       # b1
                   [0.0, 0.0],       # r2*m2
                   [0.0, 0.0],       # m2
                   [0.0, 0.0],       # I2
                   [0.0, 0.0],       # cf2
                   [0.0, 0.0]        # b2
                  ]).T

mp0 = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
##

Lastly, execute the next cell to filter the data and solve the least squares problem.

In [ ]:
mpar_opt, mpar = run_system_identification(
        measured_data_csv=measured_data_csv,
        fixed_mpar=fixed_mpar,
        variable_mpar=variable_mpar,
        mp0=mp0,
        bounds=bounds,
        optimization_method="least-squares",
        save_dir=save_dir,
        num_proc=32,
        rescale=True,
        maxfevals=100000,
        filt="butterworth")

print(mpar)

mpar.save_dict(os.path.join(save_dir, "model_parameters.yml"))

### Think-Pair-Share

Analyze the results. Is the difference between the torque predicted by the model and filtered measured torque reasonable? If not, select new bounds and initial guess for the parameters and repeat the measurement and system identification.

To test see if the results are accurate, run an experiment with the gravity compensation controller. The next cell creates a new plant with the parameters obtained through system identification and a gravity compensation controller.

In [ ]:

plant_newparams = DoublePendulumPlant(
    mass=mpar.m,
    length=mpar.l,
    com=mpar.r,
    gravity=mpar.g,
    inertia=mpar.I,
    damping=mpar.b,
    torque_limit=mpar.tl,
    coulomb_fric=mpar.cf
)

sim_newparams = Simulator(plant=plant_newparams)

gravcomp_controller = GravityCompensationController(
    mass=mpar.m,
    length=mpar.l,
    com=mpar.r,
    damping=mpar.b,
    torque_limit=mpar.tl,
    coulomb_fric=mpar.cf,
    gravity=mpar.g,
    inertia=mpar.I
)

Run the next cell to start the experiment. Try moving around the pendulum, it should hold its position. If the experiment fails, restart the kernel and run the notebook again; this time, changing the initial guess for the parameters.

In [ ]:
T_exp, X_exp, U_exp, U_des_exp = sim_newparams.run_experiment(
    tf=10,
    dt=0.002,
    controller=gravcomp_controller,
    experiment_type="DoublePendulum",
    motors = motors # [shoulder, elbow]
)